In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

to include glove file in your kernel, simple click 'input files' at the top of the notebook, and search 'glove' in the 'datasets' section.

In [ ]:
#path = '../input/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE='/content/drive/My Drive/Colab Notebooks/glove.6B.100d.txt'
TRAIN_DATA_FILE=f'/content/drive/My Drive/Colab Notebooks/User history.csv'
#TEST_DATA_FILE=f'Test2.csv'

In [ ]:
#set parameters
embed_size = 100 # how big is each word vector
max_features = 11700 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 1000 # max number of words in a comment to use

In [ ]:
#read datasets
train = pd.read_csv(TRAIN_DATA_FILE, encoding ='unicode-escape')
#test = pd.read_csv(TEST_DATA_FILE, encoding='unicode-escape')

pre-processing 

In [ ]:
list_sentences_train = train["Userhistory"].fillna("_na_").values
list_classes = ["label"]
y = train[list_classes].values
#list_sentences_test = test["Tweet"].fillna("_na_").values

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
#list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
#X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

In [ ]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

Use these vectors to create our embedding matrix, with random initialization for words that aren't in GloVe. We'll use the same mean and stdev of embeddings the GloVe has when generating the random init.

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.004451992, 0.4081574)

In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

bidirectional LSTM

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
#adam =Adam(0.001)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_t, y, batch_size=128, epochs=50 ,validation_split=0.2)

Epoch 1/50
8/8 [==============================] - 59s 7s/step - loss: 0.5660 - accuracy: 0.7606 - val_loss: 0.7376 - val_accuracy: 0.5638
Epoch 2/50
8/8 [==============================] - 59s 7s/step - loss: 0.5445 - accuracy: 0.7647 - val_loss: 0.6961 - val_accuracy: 0.5638
Epoch 3/50
8/8 [==============================] - 58s 7s/step - loss: 0.5240 - accuracy: 0.7647 - val_loss: 0.7254 - val_accuracy: 0.5638
Epoch 4/50
8/8 [==============================] - 58s 7s/step - loss: 0.5102 - accuracy: 0.7647 - val_loss: 0.7108 - val_accuracy: 0.5638
Epoch 5/50
8/8 [==============================] - 58s 7s/step - loss: 0.4985 - accuracy: 0.7647 - val_loss: 0.6696 - val_accuracy: 0.5802
Epoch 6/50
8/8 [==============================] - 58s 7s/step - loss: 0.4695 - accuracy: 0.7792 - val_loss: 0.6691 - val_accuracy: 0.6132
Epoch 7/50
8/8 [==============================] - 61s 8s/step - loss: 0.4452 - accuracy: 0.8019 - val_loss: 0.5918 - val_accuracy: 0.6872
Epoch 8/50
8/8 [==================

In [ ]:
pred = model.predict(X_t, batch_size=100, verbose=1)
print(pred)

13/13 [==============================] - 12s 908ms/step
[[9.9861836e-01]
 [7.2689608e-07]
 [9.9886405e-01]
 ...
 [1.4442447e-05]
 [1.9013229e-05]
 [9.9537468e-01]]


In [ ]:
print(min(pred))
print(max(pred))

[1.6549859e-10]
[0.99975014]


In [ ]:
np.arrays=pred
pred[pred >0.40]=int(1)
pred[pred < 0.40]=int(0)

pred.astype(int)


array([[1],
       [0],
       [1],
       ...,
       [0],
       [0],
       [1]])

In [ ]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
results = confusion_matrix(y,pred.astype(int)) 
  
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(y, pred.astype(int) ))
print ('Report : ')
print (classification_report(y,pred.astype(int) ))





Confusion Matrix :
[[873   5]
 [ 75 259]]
Accuracy Score : 0.933993399339934
Report : 
              precision    recall  f1-score   support

           0       0.92      0.99      0.96       878
           1       0.98      0.78      0.87       334

    accuracy                           0.93      1212
   macro avg       0.95      0.88      0.91      1212
weighted avg       0.94      0.93      0.93      1212

